In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, time, json
import pandas as pd
from openai import OpenAI
from tqdm.auto import tqdm
import spacy
import pandas as pd

import sys
sys.path.append("../")
import os
from src.functional import free_gpu_cache

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.models import load_LM
import torch
from nnsight import LanguageModel
from src.utils import env_utils
from src.functional import free_gpu_cache

# model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
model_key = "meta-llama/Meta-Llama-3-70B-Instruct"

lm = load_LM(
    model_key=model_key,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 30/30 [01:07<00:00,  2.25s/it]

loaded /home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-70B-Instruct | size: 36650.535 MB


In [4]:
from src.models import reset_forwards

In [141]:
from src.dataset import load_worldstate_dataset
from src.functional import predict_next_token 

dataset = load_worldstate_dataset()

In [154]:
#! why the experiment doesn't work if b is the correct answer?
reset_forwards(lm)
correct_ans_idx = 0
question, answer = dataset.__getitem__(22, tags=("a", "b"), correct_ans_idx=correct_ans_idx)
print(question)
print(answer)

Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option (either a or b) after the "Answer:" tag.

Story: Luka is a gardener working in a beautiful park in the middle of the city. Luka wants to water the plants in the park to ensure they stay healthy and vibrant. The sky is cloudy, and Luka assumes that it might rain soon. However, the wind picks up and blows the clouds away, revealing a bright and sunny sky. Luka does not notice the clouds clearing and the sun shining.

Question: Does Luka believe that it will rain soon, or does he think the weather has become sunny?
a) Luka believes the weather has become sunny.
b) Luka believes that it will rain soon.
Answer:
b


In [155]:
# from src.functional import predict_next_token
# predict_next_token(
#     lm = lm,
#     input = question
# )

In [156]:
free_gpu_cache()

In [157]:
from src.models import prepare_input
from src.functional import find_token_range
input = prepare_input(
    prompts = question,
    tokenizer=lm,
    return_offsets_mapping=True,
)

option_1_ends = find_token_range(
    string=question,
    substring = ".\n",
    tokenizer=lm,
    occurrence=-2,
    offset_mapping=input["offset_mapping"][0],
)[1] - 1

option_2_ends = find_token_range(
    string=question,
    substring = ".\n",
    tokenizer=lm,
    occurrence=-1,
    offset_mapping=input["offset_mapping"][0],
)[1] - 1

input.pop("offset_mapping")
print(option_1_ends, lm.tokenizer.decode(input["input_ids"][0][option_1_ends]))
print(option_2_ends, lm.tokenizer.decode(input["input_ids"][0][option_2_ends]))

178 .

189 .



In [159]:
print(f"\"{lm.tokenizer.decode(input['input_ids'][0][:option_1_ends])}\"")

"<|begin_of_text|>Instruction: Keep track of people's knowledge defined in the story. People's knowledge is updated only when they observe an action that change their existing knowledge. To answer the question following the story, choose the correct option by predicting the answer option (either a or b) after the "Answer:" tag.

Story: Luka is a gardener working in a beautiful park in the middle of the city. Luka wants to water the plants in the park to ensure they stay healthy and vibrant. The sky is cloudy, and Luka assumes that it might rain soon. However, the wind picks up and blows the clouds away, revealing a bright and sunny sky. Luka does not notice the clouds clearing and the sun shining.

Question: Does Luka believe that it will rain soon, or does he think the weather has become sunny?
a) Luka believes the weather has become sunny"


In [160]:
# from src.hooking.llama_attention import LlamaAttentionPatcher, KQV_patch
# import types
# import baukit
# from src.functional import interpret_logits


# free_gpu_cache()

# patch_range = list(range(35, 39))
# module_name_format = "model.layers.{}.self_attn"

# kqv_states = {layer_idx : {} for layer_idx in patch_range}
# kqv_spec = (
#     [(h_idx, option_1_ends) for h_idx in range(lm.config.num_attention_heads)] +
#     [(h_idx, option_2_ends) for h_idx in range(lm.config.num_attention_heads)]
# ) 

# reset_forwards(lm)
# for layer_idx in patch_range:
#     attn_module = baukit.get_module(
#         lm._model, module_name_format.format(layer_idx)
#     )
#     attn_module.forward = types.MethodType(
#         LlamaAttentionPatcher(
#             block_name=module_name_format.format(layer_idx),
#             save_kqv_for=kqv_spec,
#             kqv_states=kqv_states[layer_idx],
#         ), attn_module
#     )
    
# with torch.inference_mode():
#     with lm.trace(input, scan=False, validate=False) as trace:
#         logits = lm.output.logits[0, -1, :].save()

# free_gpu_cache()
# reset_forwards(lm)

# interpret_logits(
#     logits=logits,
#     tokenizer=lm
# )

In [161]:
free_gpu_cache()

In [162]:
# kqv_patches = {layer_idx : {} for layer_idx in patch_range}
# for layer_idx in kqv_states:
#     for h_idx in range(lm.config.num_attention_heads):
#         kqv_2 = kqv_states[layer_idx][(h_idx, option_2_ends)]
#         kqv_patches[layer_idx][(h_idx, option_1_ends)] = KQV_patch(
#             # k = kqv_2["key"],
#             v = kqv_2["value"],
#         )

#         kqv_1 = kqv_states[layer_idx][(h_idx, option_1_ends)]
#         kqv_patches[layer_idx][(h_idx, option_2_ends)] = KQV_patch(
#             # k = kqv_1["key"],
#             v = kqv_1["value"],
#         )
    

# reset_forwards(lm)
# for layer_idx in patch_range:
#     attn_module = baukit.get_module(
#         lm._model, module_name_format.format(layer_idx)
#     )
#     attn_module.forward = types.MethodType(
#         LlamaAttentionPatcher(
#             block_name=module_name_format.format(layer_idx),
#             kqv_patches=kqv_patches[layer_idx],
#         ), attn_module
#     )
# with torch.inference_mode():
#     with lm.trace(input, scan=False, validate=False) as trace:
#         logits = lm.output.logits[0, -1, :].save()

# reset_forwards(lm)
# free_gpu_cache()

# interpret_logits(
#     logits=logits,
#     tokenizer=lm
# )

In [163]:
from src.functional import get_module_nnsight, interpret_logits

patch_range = list(range(33, 40))
attn_module_name_format = "model.layers.{}.self_attn"
kqv_states = {layer_idx : {} for layer_idx in patch_range}

with torch.inference_mode():
    with lm.trace(input, scan=False, validate=False) as trace:
        for layer_idx in patch_range:
            module = get_module_nnsight(lm, attn_module_name_format.format(layer_idx))
            kqv_states[layer_idx] = {
                "q" : module.q_proj.output.clone().save(),
                "k" : module.k_proj.output.clone().save(),
                "v" : module.v_proj.output.clone().save(),
            }

            # option_a_key = lm.model.layers[layer_idx].self_attn.k_proj.output[0, option_1_ends].clone()
            # option_a_value = lm.model.layers[layer_idx].self_attn.v_proj.output[0, option_1_ends].clone()
            # option_b_key = lm.model.layers[layer_idx].self_attn.k_proj.output[0, option_2_ends].clone()
            # option_b_value = lm.model.layers[layer_idx].self_attn.v_proj.output[0, option_2_ends].clone()

            # # model.model.layers[layer_idx].self_attn.k_proj.output[0, period_token_indices[0]] = option_b_key
            # lm.model.layers[layer_idx].self_attn.v_proj.output[0, option_1_ends] = option_b_value * 1
            # # model.model.layers[layer_idx].self_attn.k_proj.output[0, period_token_indices[1]] = option_a_key
            # lm.model.layers[layer_idx].self_attn.v_proj.output[0, option_2_ends] = option_a_value * 1
        
        logits = lm.output.logits[0, -1, :].save()

interpret_logits(tokenizer = lm, logits = logits)

[PredictedToken(token=' b', prob=0.9983530044555664, logit=22.546875, token_id=293),
 PredictedToken(token=' ', prob=0.0003482514584902674, logit=14.5859375, token_id=220),
 PredictedToken(token=' **', prob=0.00023380493803415447, logit=14.1875, token_id=3146),
 PredictedToken(token='\xa0b', prob=0.00011941587581532076, logit=13.515625, token_id=115648),
 PredictedToken(token=' Instruction', prob=0.00011394743341952562, logit=13.46875, token_id=30151)]

In [164]:
kqv_states[35]["k"].shape

torch.Size([1, 192, 1024])

In [165]:
with torch.inference_mode():
    with lm.trace(input, scan=False, validate=False) as trace:
        for layer_idx in patch_range:
            module = get_module_nnsight(lm, attn_module_name_format.format(layer_idx))

            # module.q_proj.output[:, -1, :] = kqv_states[layer_idx]["q"][:, -1, :]
            # module.q_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["q"][:, option_1_ends, :]

            # module.k_proj.output[:, option_1_ends, :] = kqv_states[layer_idx]["k"][:, option_2_ends, :]
            # module.k_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["k"][:, option_1_ends, :]

            module.v_proj.output[:, option_1_ends, :] = kqv_states[layer_idx]["v"][:, option_2_ends, :]
            module.v_proj.output[:, option_2_ends, :] = kqv_states[layer_idx]["v"][:, option_1_ends, :]

        logits = lm.output.logits[0, -1, :].save()

interpret_logits(tokenizer = lm, logits = logits)

[PredictedToken(token=' b', prob=0.7647252082824707, logit=23.15625, token_id=293),
 PredictedToken(token=' a', prob=0.23322799801826477, logit=21.96875, token_id=264),
 PredictedToken(token=' ', prob=0.0007308052154257894, logit=16.203125, token_id=220),
 PredictedToken(token=' \n', prob=0.00023911852622404695, logit=15.0859375, token_id=720),
 PredictedToken(token=' **', prob=0.00019979098578915, logit=14.90625, token_id=3146)]